In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import time
from astropy.stats import sigma_clip
from transitleastsquares import (
    transitleastsquares,
    cleaned_array,
    catalog_info,
    transit_mask
    )
import pandas as pd
from OrionPY import Tools



In [2]:
# Loading in the original data for obj_ids
hdu = Tools()





/home/jpr64/orion/Orion_main/OrionPY.py:71: RuntimeWarning: invalid value encountered in log10
  self.mag = -2.5*np.log10(self.flux)


In [3]:
data = np.load('/data/jpr64/rebinned_data_test.npy', allow_pickle = True)
lc = [x for x in data]
obj_ids = [x for x in hdu.hdu[1].data['obj_id']]



In [4]:
# Calculating magnitudes
mean_mag = np.zeros(np.size(lc))

for i, j in enumerate(lc):
        magnitudes = -2.5 * np.log10(j[:,1]) + 20.2
        mean_mag[i] = np.nanmean(magnitudes)
        
        
        
        
        

/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


In [5]:
mag_min = 8.5
mag_max = 15

index_keep = np.where((mean_mag > mag_min) & (mean_mag< mag_max))



In [7]:
# Keeping only stars with mags above 8.5 and below 15 for later comparison
# index_keep = np.where((mean_mag > 8.5) & (mean_mag<15))
index_keep = index_keep[0]

lc  = [lc[x] for x in index_keep]
obj_ids = [obj_ids[x] for x in index_keep]


In [8]:
output = pd.DataFrame()

###
for i, j in enumerate(lc):
    if i == 3: break
    time, flux = cleaned_array(j[:,0], j[:,1])
    flux /= np.nanmean(flux)
    sigma_clipped = sigma_clip(flux, sigma_lower = float('inf'), sigma_upper= 4)
    
    if np.size(time) < 100: continue # TLS fails if the array only has a few values in it. Anything less than 100 wouldn't be useful anyway.  
   
    model = transitleastsquares(time, flux)
    results = model.power(period_min = 1, period_max = 11, use_threads = 40)
    
    results["obj_id"] = obj_ids[i]
    output = output.append(results, ignore_index=True)
   
    
    
    
    

Transit Least Squares TLS 1.0.26 (20 March 2021)


/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/transitleastsquares/transit.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  lc_arr = numpy.array(lc_arr)
  0%|          | 0/10003 periods | 00:00<?

Creating model cache for 36 durations
Searching 3937 data points, 10003 periods from 1.0 to 10.997 days
Using 40 of 64 CPU threads


100%|██████████| 10003/10003 periods | 00:34<00:00
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/transitleastsquares/main.py:404: UserWarning: 63 of 76 transits without data. The true period may be twice the given period.
  warnings.warn(text)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/transitleastsquares/transit.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-

Transit Least Squares TLS 1.0.26 (20 March 2021)
Creating model cache for 36 durations
Searching 4006 data points, 10524 periods from 1.0 to 10.998 days
Using 40 of 64 CPU threads


100%|██████████| 10524/10524 periods | 00:31<00:00
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jpr64/.conda/envs/orion_jake/lib/python3.7/site-packages/transitleastsquares/main.py:404: UserWarning: 49 of 59 transits without data. The true period may be twice the given period.
  warnings.warn(text)


In [14]:
output.to_pickle('/data/jpr64/output_test.pkl')

In [15]:
reading = pd.read_pickle('/data/jpr64/output_test.pkl')

,FAP,SDE,SDE_raw,SR,T0,after_transit_count,before_transit_count,chi2,chi2_min,chi2red,...,power,power_raw,rp_rs,snr,snr_per_transit,snr_pink_per_transit,transit_count,transit_depths,transit_depths_uncertainties,transit_times
0,0.006483,7.225084,6.108805,"[0.9372557276073122, 0.9370453250040619, 0.939...",1333.190249,72.0,70.0,"[3873.387851420103, 3874.25757550451, 3865.484...",3630.354949,"[0.9848430845207482, 0.9850642195536512, 0.982...",...,"[-2.1867212889782732, -2.2195713355777693, -1....","[-0.15036530260519937, -0.17135440176540068, 0...",0.065237,7.888991,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",76.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1333.1902485072694, 1335.2043313835738, 1337...."
1,0.012485,6.833969,5.044323,"[0.9553377545321289, 0.9551311944205402, 0.954...",1326.724024,31.0,39.0,"[3909.884546241071, 3910.7301119527638, 3912.1...",3735.260323,"[0.9769826452376489, 0.9771939310226796, 0.977...",...,"[-0.8095679941001539, -0.8398097012676873, -0....","[0.5007235587064709, 0.4797096951321366, 0.444...",0.177596,7.300697,"[0.0, 2.386560939237457, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59.0,"[nan, 0.9445661246520263, nan, nan, nan, nan, ...","[nan, 0.01793650580633996, nan, nan, nan, nan,...","[1326.7240240195697, 1329.4146120429407, 1332...."
